In [13]:
from os import listdir
import pandas as pd 

In [27]:
df = pd.read_csv('pct_change_stocks.csv')

for i in os.listdir():
    if 'csv' in i: 
        df = df.append(pd.read_csv(i))
    
df = df.set_index('date')

In [33]:
df.describe()

,time_delta,pct_change
count,831.000000,831.000000
mean,14.302046,8.316644
std,10.376246,11.548406
min,1.000000,-11.067195
25%,4.000000,1.828057
50%,14.000000,4.595187
75%,26.000000,10.207615
max,29.000000,85.959878


In [53]:
df['norm_pct_change'] = df['pct_change'].div(df['pct_change'].max())
df['norm_time_delta'] = df['time_delta'].div(df['time_delta'].max())

In [55]:
df[['content','norm_pct_change', 'norm_time_delta']]

,content,norm_pct_change,norm_time_delta
date,,,
"December 19, 2018","100 days to Brexit, EU tells L...",-0.021475,0.482759
"May 15, 2020",Foxconn quarterly profit hits ...,0.170619,0.896552
"August 31, 2018",Wall Street mixed as U.S.-Cana...,0.003731,0.137931
"January 30, 2019",iPhone suppliers rise after Ap...,0.067723,0.965517
"June 3, 2019",Apple dips on news of DoJ anti...,0.197559,1.000000
...,...,...,...
"March 5, 2020",Russia's car sales fall 2.2% i...,-0.003776,0.034483
"April 8, 2020",Russia car sales in March up 4...,0.065728,0.206897
"May 12, 2020",Russia car sales plunge 72.4% ...,0.120550,0.793103


In [64]:
df.content.map(len)
#do i need to add padding to even out the length? YES

date
December 19, 2018    5515
May 15, 2020         4152
August 31, 2018      4271
January 30, 2019     2115
June 3, 2019         1956
                     ... 
March 5, 2020        1465
April 8, 2020        1546
May 12, 2020         1897
July 7, 2020         1462
July 24, 2018        2062
Name: content, Length: 831, dtype: int64

In [65]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [70]:
analyzer = SentimentIntensityAnalyzer()
sentiment = df.content.apply(analyzer.polarity_scores).tolist()

In [86]:
compound = [i['compound'] for i in sentiment]

-0.9997

In [89]:
df['sentiment'] = compound

date,"December 19, 2018","May 15, 2020","August 31, 2018","January 30, 2019","June 3, 2019"
ticker,aapl,aapl,aapl,aapl,aapl
content,"100 days to Brexit, EU tells L...",Foxconn quarterly profit hits ...,Wall Street mixed as U.S.-Cana...,iPhone suppliers rise after Ap...,Apple dips on news of DoJ anti...
time_delta,14,26,4,28,29
pct_change,-1.84598,14.6664,0.320694,5.82148,16.9821
norm_pct_change,-0.0214749,0.170619,0.00373074,0.0677232,0.197559
norm_time_delta,0.482759,0.896552,0.137931,0.965517,1
sentiment,-0.2236,0.8018,0.9928,0.8608,0.6705


In [92]:
df.head()

,ticker,content,time_delta,pct_change,norm_pct_change,norm_time_delta,sentiment
date,,,,,,,
"December 19, 2018",aapl,"100 days to Brexit, EU tells L...",14,-1.845982,-0.021475,0.482759,-0.2236
"May 15, 2020",aapl,Foxconn quarterly profit hits ...,26,14.666409,0.170619,0.896552,0.8018
"August 31, 2018",aapl,Wall Street mixed as U.S.-Cana...,4,0.320694,0.003731,0.137931,0.9928
"January 30, 2019",aapl,iPhone suppliers rise after Ap...,28,5.821480,0.067723,0.965517,0.8608
"June 3, 2019",aapl,Apple dips on news of DoJ anti...,29,16.982107,0.197559,1.000000,0.6705


In [94]:
from nltk.corpus import stopwords
import string

In [132]:
def clean_doc(content): 
    tokens = content.split()
    table = str.maketrans('','', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    tokens = [word for word in tokens if len(word)>1]
    return tokens 

In [136]:
clean_content = [clean_doc(df.content[i]) for i in range(len(df.content))]

In [146]:
map(clean_content, len)

TypeError: 'builtin_function_or_method' object is not iterable

In [141]:
maximum

4766